In [20]:
import pandas as pd
import numpy as np

In [49]:
df=pd.read_csv("NED26.05.1-D-12.1.0-20160501.csv",skiprows=12)

In [50]:
df.head()

,Exclusion Code,D,G,Galaxy ID,m-M,err,D (Mpc),Method,REFCODE,SN ID,redshift (z),Hubble const.,Adopted LMC modulus,Date (Yr. - 1980),Notes
0,RN,1,1,SDSS-II SN 13651,41.64,0.17,1700,SNIa SDSS,2014arXiv1401.3317S,SDSS-II SN 13651,0.249917,70,NaN,34,SDSS CAND. AGN; MLCS2K2
1,RN,2,1,SDSS-II SN 13651,42.10,0.21,2110,SNIa SDSS,2014arXiv1401.3317S,SDSS-II SN 13651,0.249917,70,NaN,34,SDSS CAND. AGN; SALT 2
2,RN,999999,1,SDSS-II SN 13651,41.64,0.17,1700,SNIa SDSS,2014arXiv1401.3317S,SDSS-II SN 13651,0.249917,70,NaN,34,"SDSS-II SN 13651, SDSS CAND. AGN; MLCS2K2"
3,RN,999999,1,SDSS-II SN 13651,42.10,0.21,2110,SNIa SDSS,2014arXiv1401.3317S,SDSS-II SN 13651,0.249917,70,NaN,34,"SDSS-II SN 13651, SDSS CAND. AGN; SALT 2"
4,NaN,3,2,UGC 12889,34.16,0.40,68,Tully-Fisher,2007A&A...465...71T,NaN,NaN,NaN,NaN,27,mean


In [58]:
listrs=np.unique(df['Galaxy ID'][np.isnan(df['redshift (z)'])])

In [59]:
from astroquery.ned import Ned

In [157]:
id=150
listrs[id]
result_table = Ned.query_object(listrs[id])
result_table['RA(deg)']

7.1115300000000001


In [154]:
result_table.columns

<TableColumns names=('No.','Object Name','RA(deg)','DEC(deg)','Type','Velocity','Redshift','Redshift Flag','Magnitude and Filter','Distance (arcmin)','References','Notes','Photometry Points','Positions','Redshift Points','Diameter Points','Associations')>

In [146]:
names=[]
rshifts=[]
k=0
for i in listrs:
    k=k+1
    result_table = Ned.query_object(i)
    if(not(result_table['Redshift'][0] is np.ma.masked)):
        print k,i, result_table['Redshift'][0]
        names.append(i)
        rshifts.append(result_table['Redshift'][0])

20 1SAX J1338.1-8030 1.6187
28 2MASS J00333438-1921331 0.61
29 2MASS J01145278+1325376 2.025
31 2MASS J06504649+2502596 0.203
32 2MASS J08162719-1311524 0.045978
33 2MASS J08442426+1245464 2.4934
35 2MASS J17250434+1152155 0.018
39 2MASX J00001440-7652248 0.053267
40 2MASX J00002213-0137463 0.038283
41 2MASX J00004283-7217148 0.034757
42 2MASX J00005579-2554210 0.050402
43 2MASX J00010098-4345156 0.03926
44 2MASX J00010107-4351356 0.039321
45 2MASX J00013295-0726099 0.029474
46 2MASX J00021772-4345168 0.040912
47 2MASX J00021977-4352198 0.038583
48 2MASX J00022478-5445592 0.032576
49 2MASX J00024069+0846251 0.040301
50 2MASX J00024429-5344549 0.034887
51 2MASX J00030547-5402112 0.036212
52 2MASX J00034674-5514189 0.033353
53 2MASX J00035182-1206094 0.04339
54 2MASX J00035710-6146566 0.029717
55 2MASX J00040601-5134568 0.034357
56 2MASX J00041398-5113247 0.037129
57 2MASX J00041814-2506060 0.050482
58 2MASX J00042841-5254136 0.030828
59 2MASX J00050371-5520325 0.03802
60 2MASX J00050534

KeyboardInterrupt: 

In [121]:
if(not(result_table['Redshift'][0] is np.ma.masked)):
    print 'hey'

hey


In [103]:
A=np.isfinite(result_table['Redshift'][0])

In [104]:
A.mask==True

AttributeError: 'numpy.bool_' object has no attribute 'mask'

In [94]:
A==masked

NameError: name 'masked' is not defined